#### VactionPy

##### Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_key_main import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,gwadar,25.1216,62.3254,24.52,67,79,1.35,PK,1699407388
1,1,papatowai,-46.5619,169.4708,12.10,53,91,4.37,NZ,1699407184
2,2,adamstown,-25.0660,-130.1015,22.71,86,100,8.65,PN,1699407389
3,3,qaqortoq,60.7167,-46.0333,1.84,59,100,1.31,GL,1699407174
4,4,bethel,41.3712,-73.4140,14.05,61,0,3.09,US,1699407389


##### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# %%capture --no-display
# Configure the map plot_1
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500, 
    size = "Humidity", 
    scale = 1,
    color = "City",
    alpha = 0.5  
)

# Display the map
map_plot_1

c:\Users\David Rios\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

##### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_ideal_df = city_data_df
#Ideal conditions Max Temp > 21C, Max Temp < 27C, Wind Speed < 4.5 m/s, and 0 cloudiness
city_ideal_df = city_ideal_df.loc[(city_ideal_df["Max Temp"] > 21) & (city_ideal_df["Max Temp"] < 27)]
city_ideal_df = city_ideal_df.loc[city_ideal_df["Wind Speed"] < 4.5]
city_ideal_df = city_ideal_df.loc[city_ideal_df["Cloudiness"] == 0]


# Drop any rows with null values
city_ideal_df = city_ideal_df.dropna()

city_ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
57,57,brak,27.5496,14.2714,21.36,24,0,3.51,LY,1699407405
205,205,mancio lima,-7.6142,-72.8958,26.04,94,0,0.00,BR,1699407451
232,232,rizokarpaso,35.6000,34.3750,21.80,78,0,2.06,CY,1699407460
456,456,taoudenni,22.6783,-3.9836,21.76,25,0,4.04,ML,1699407528
489,489,sao vicente,-23.9631,-46.3919,21.82,26,0,1.23,BR,1699407539
545,545,salalah,17.0151,54.0924,25.05,83,0,2.06,OM,1699407557


##### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(city_ideal_df, columns = ["City", "Country", "Lat", "Lng", "Humidity", "Hotel Name"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
57,brak,LY,27.5496,14.2714,24,
205,mancio lima,BR,-7.6142,-72.8958,94,
232,rizokarpaso,CY,35.6000,34.3750,78,
456,taoudenni,ML,22.6783,-3.9836,25,
489,sao vicente,BR,-23.9631,-46.3919,26,


##### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000 
params = {
        "categories" : "accommodation.hotel",
        "apiKey" : geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")   
# Display sample data
hotel_df

Starting hotel search
brak - nearest hotel: فندق براك السياحي
mancio lima - nearest hotel: No hotel found
rizokarpaso - nearest hotel: Villa Carparis
taoudenni - nearest hotel: No hotel found
sao vicente - nearest hotel: Pousada Vitória
salalah - nearest hotel: Muscat International Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
57,brak,LY,27.5496,14.2714,24,فندق براك السياحي
205,mancio lima,BR,-7.6142,-72.8958,94,No hotel found
232,rizokarpaso,CY,35.6000,34.3750,78,Villa Carparis
456,taoudenni,ML,22.6783,-3.9836,25,No hotel found
489,sao vicente,BR,-23.9631,-46.3919,26,Pousada Vitória
545,salalah,OM,17.0151,54.0924,83,Muscat International Hotel


##### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# %%capture --no-display

map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500, 
    size = "Humidity", 
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"],
    alpha = 0.5  
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)